In [ ]:
import intake
df = intake.open_csv('./data/bird_migration/{species}.csv').read()

## HvPlot

Now that our data are loaded in, we can use `hvplot` to look directly at the data. `hvplot` provides an interface that is very similar to `.plot` but the output is an interactive and alterable object.

In [ ]:
import pandas as pd
import hvplot.pandas

In [ ]:
df.hvplot.points('lon', 'lat', height=500)

It's kind of hard to see all those dots.

In [ ]:
df.hvplot.points('lon', 'lat', height=500, datashade=True)

Can we use color more effectively?

In [ ]:
df.hvplot.points('lon', 'lat', height=500, color='day', cmap='colorwheel', size=1)

How can we tell which species is which?

In [ ]:
df.hvplot.points('lon', 'lat', height=500, color='day', cmap='colorwheel', size=1, hover_cols=['species'])

Now we effectively have our whole dataset in one plot and we can use interactive tools to explore. This is often enough for initial stages of data exploration. But if you are interested in looking more closely at particular species, you can use grouping.

## Grouping

In [ ]:
p = df.hvplot.points('lon', 'lat', height=500, color='day', cmap='colorwheel', groupby='species')
p

In [ ]:
print(p)

We can inspect just one bird in the same way that we would use a dictionary:

In [ ]:
bird = 'Connecticut_Warbler'
p[bird]

Notice the gap between days 77 and 104. To clean up the data a little, we'll just do a linear interpolation between consecutive days.

In [ ]:
def fill_day(v):
    next_year = v.assign(day=v.day + v.day.max())
    last_year = v.assign(day=v.day - v.day.max())
    surrounding_years = pd.concat([last_year, v, next_year])
    filled = surrounding_years.assign(
        lat=surrounding_years.lat.interpolate(), 
        lon=surrounding_years.lon.interpolate())
    this_year = filled[filled.day.isin(v.day)]
    return this_year

In [ ]:
filled_days = pd.concat([fill_day(v) for k, v in df.groupby('species')])

Now let's check on that Connecticut Warbler again

In [ ]:
filled_days.hvplot.points('lon', 'lat', height=500, color='day', cmap='colorwheel', groupby='species').select(species=bird)

## Combining plots

We can combine different plots to explore along various axes at the same time.

In [ ]:
day_p = filled_days.hvplot.points('lon', 'lat', color='day', cmap='colorwheel', groupby='species')
lat_p = filled_days.hvplot('day', 'lat', groupby='species').hist() 
(day_p + lat_p).cols(1)

[Next Section](./02_holoviews.ipynb#Holoviews)